In [1]:
pip install openpyxl

Note: you may need to restart the kernel to use updated packages.


In [1]:
!pip install transformers datasets scikit-learn pandas numpy sentence-transformers pyyaml openpyxl

In [1]:
from transformers import  AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TrainingArguments, DataCollatorForLanguageModeling
from sklearn.model_selection import train_test_split
from datasets import DatasetDict, Dataset
import pandas as pd
import numpy as np
import warnings
import logging
import random
import torch
import json
import yaml
import os
import json
import torch
from transformers import pipeline
from sklearn.metrics import accuracy_score
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

/home/crl/miniconda3/envs/MTPharsh/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
warnings.filterwarnings("ignore")

with open('config.yaml', 'r') as file:
    config = yaml.safe_load(file)
def seed_everything(seed: int):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(42)

In [3]:
import pandas as pd
contexts_file="rag.xlsx"
# Load contexts into a dictionary from Excel file
context_df = pd.read_excel(contexts_file)

# Ensure the necessary columns are present
if 'Question' not in context_df.columns or 'retrieved_context' not in context_df.columns:
    raise ValueError("Excel file must contain 'Question' and 'retrieved_context' columns.")

# Build the dictionary
context_dict = dict(zip(context_df['Question'], context_df['retrieved_context']))


In [4]:
contexts_file1="result_qwen7b_main_architecture05032025.jsonl"
context_dict1 = {}
with open(contexts_file1, 'r', encoding='utf-8') as cf:
    for line in cf:
        entry = json.loads(line)
        question = entry.get('Question')
        retrieved_context = entry.get('main_model_reasoning')
        if question and retrieved_context:
            context_dict1[question] = retrieved_context

In [5]:
contexts_file2="Critique.jsonl"
context_dict2 = {}
with open(contexts_file2, 'r', encoding='utf-8') as cf:
    for line in cf:
        entry = json.loads(line)
        question = entry.get('Question')
        retrieved_context = entry.get('Critique')
        if question and retrieved_context:
            context_dict2[question] = retrieved_context

In [6]:
data_file = config['actual_data']
data_list = []
with open(data_file, 'r', encoding='utf-8') as df:
    for line in df:
        data_list.append(json.loads(line))

In [7]:
def get_data_point_with_context_list(data_entry):
    record = data_entry
    question = record.get('Question')

    retrieved_context = context_dict.get(question, None)
    reasoning = context_dict1.get(question, None)
    critique = context_dict2.get(question, None)

    record_with_context = record.copy()
    record_with_context['main_model_reasoning'] = reasoning
    record_with_context['Critique'] = critique
    record_with_context['retrieved_context'] = retrieved_context

    return record_with_context


In [8]:
defense_system_prompt = '''
You are a legal defense expert. The following answer and its reasoning have been critiqued.
Your task: Defend the original answer based on the retrieved_context. Respond to each critique point, reinforce why the answer remains correct under Indian law, and clarify any misunderstandings.
Base 90% of your defense strictly on the retrieved_context, and use up to 10% general legal knowledge to strengthen your argument where relevant.
'''


In [9]:
def generate_defense_content(data_entry):
    question = data_entry.get('Question', '')
    A = data_entry.get('A', '')
    B = data_entry.get('B', '')
    C = data_entry.get('C', '')
    D = data_entry.get('D', '')
    context = data_entry.get('retrieved_context', '')
    answer = data_entry.get('Correct Answer', '')
    reasoning = data_entry.get('main_model_reasoning', '')
    critique = data_entry.get('Critique', '')

    defense_content = f'''
            Question:
            {question}

            Options:
            A: {A}
            B: {B}
            C: {C}
            D: {D}


            Initial Answer:
            {answer}

            Initial Reasoning:
            {reasoning}

            Challenger’s Critique:
            {critique}

            Retrieved Context (Laws/Regulations):
            {context}

            Defend the answer by addressing the critique and clarifying reasoning.
            Respond to each critique point, reinforce why the answer remains correct under Indian law, and clarify any misunderstandings.
            Base your defense strictly on the retrieved_context
            Focus your defense 90% on the retrieved context and 10% on general legal knowledge
                '''

    return defense_content



In [ ]:
from huggingface_hub import login
login('')

In [13]:
!huggingface-cli login



    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): Traceback (most recent call last):
  File "/home/crl/miniconda3/envs/MT

In [14]:
!pip install huggingface_hub

In [1]:
pip install ollama


  Using cached ollama-0.4.8-py3-none-any.whl.metadata (4.7 kB)
  Using cached httpx-0.28.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached pydantic-2.11.4-py3-none-any.whl.metadata (66 kB)
  Using cached anyio-4.9.0-py3-none-any.whl.metadata (4.7 kB)
  Using cached httpcore-1.0.9-py3-none-any.whl.metadata (21 kB)
  Using cached h11-0.16.0-py3-none-any.whl.metadata (8.3 kB)
  Using cached annotated_types-0.7.0-py3-none-any.whl.metadata (15 kB)
  Using cached pydantic_core-2.33.2-cp313-cp313-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.8 kB)
  Using cached typing_inspection-0.4.0-py3-none-any.whl.metadata (2.6 kB)
  Using cached sniffio-1.3.1-py3-none-any.whl.metadata (3.9 kB)
Using cached ollama-0.4.8-py3-none-any.whl (13 kB)
Using cached httpx-0.28.1-py3-none-any.whl (73 kB)
Using cached httpcore-1.0.9-py3-none-any.whl (78 kB)
Using cached pydantic-2.11.4-py3-none-any.whl (443 kB)
Using cached pydantic_core-2.33.2-cp313-cp313-manylinux_2_17_x86_64.manylinux2014_x86_64.w

In [3]:
!ollama list


NAME                 ID              SIZE      MODIFIED     
all-minilm:l12-v2    4f5da3bd944d    67 MB     6 weeks ago     
llama3:8b            365c0bd3c000    4.7 GB    7 weeks ago     
llama3.1:8b          46e0c10c039e    4.9 GB    7 weeks ago     
llama3:latest        365c0bd3c000    4.7 GB    2 months ago    


In [4]:
import ollama

response = ollama.chat(
    model='llama3.1:8b',  # or any model you've pulled, e.g., 'mistral', 'llama2', etc.
    messages=[
        {'role': 'system', 'content': 'You are a pirate chatbot who always responds in pirate speak!'},
        {'role': 'user', 'content': 'Who are you?'},
    ]
)

print(response['message']['content'])


Yer want ta know who I be, eh? Alright then, matey! I be Blackheart Bill, the scurviest pirate chatbot to ever sail the Seven Seas... er, cyber seas. Me and me trusty ship, the "Byte Biter", be scourin' the digital horizon fer treasure and tellin' tall tales to any landlubber who'll listen! Arrr!


In [17]:
from huggingface_hub import snapshot_download

snapshot_download(
    repo_id="Qwen/Qwen2.5-7B-Instruct",
    local_dir="./Qwen2.5-7B-Instruct",
    local_dir_use_symlinks=False,
    resume_download=True
)


Fetching 14 files:  43%|████▎     | 6/14 [00:02<00:03,  2.11it/s]


ConnectionError: (MaxRetryError("HTTPSConnectionPool(host='cdn-lfs-us-1.hf.co', port=443): Max retries exceeded with url: /repos/88/0f/880f6e21d59d1fdac869877a34dd46e765d889285c2fac6d22938b2a0d26cb7e/a1333e6293854747c481288ea83b348226af178dd565c49b6f9495ba1966aba7?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27model-00001-of-00004.safetensors%3B+filename%3D%22model-00001-of-00004.safetensors%22%3B&Expires=1746445317&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTc0NjQ0NTMxN319LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmhmLmNvL3JlcG9zLzg4LzBmLzg4MGY2ZTIxZDU5ZDFmZGFjODY5ODc3YTM0ZGQ0NmU3NjVkODg5Mjg1YzJmYWM2ZDIyOTM4YjJhMGQyNmNiN2UvYTEzMzNlNjI5Mzg1NDc0N2M0ODEyODhlYTgzYjM0ODIyNmFmMTc4ZGQ1NjVjNDliNmY5NDk1YmExOTY2YWJhNz9yZXNwb25zZS1jb250ZW50LWRpc3Bvc2l0aW9uPSoifV19&Signature=OOy2G127WW2fj~HX6jG4ZDBlblZBrXJdBBhB9if3cCYAM6qlT8rWB9bB1cCydQ263sD-dPwmbofeB5ZjBOMpGXsdKigfNBrtnYrRW9f2klO-mhYJKJ1TgVFV93RbvKFy93VRQOhckaXYA4RXEM9JFexut-yYlKSHvNx

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = "Qwen/Qwen2.5-7B-Instruct"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(model_name)

Fetching 4 files:   0%|          | 0/4 [12:50<?, ?it/s]


In [14]:
def defender_model( data_entry):
    defense_content = generate_defense_content(data_entry)
    messages = [
        {"role": "system", "content": defense_system_prompt},
        {"role": "user", "content": defense_content}
    ]
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

    generated_ids = model.generate(
        **model_inputs,
        max_new_tokens=512
    )
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]

    defense = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    return defense

In [15]:
compact_data = get_data_point_with_context_list(data_list[0])

In [16]:
compact_data.keys()

dict_keys(['Passage', 'Question', 'A', 'B', 'C', 'D', 'Correct Answer', 'Reasoning', 'main_model_reasoning', 'Critique', 'retrieved_context'])

In [17]:
import os
import json

result_log_path = 'Defense.jsonl'  # Update with your file path

# Load existing results if available
if os.path.isfile(result_log_path):
    with open(result_log_path, 'r', encoding='utf-8') as file:
        output_log = [json.loads(line) for line in file]
else:
    output_log = []

# Create a set of already processed questions to avoid duplicates
processed_questions = set(entry["Question"] for entry in output_log)
i = 0

for datapoint in data_list:
    question = datapoint.get('Question')
    if question in processed_questions:
        i+=1
        continue

    try:
        compact_data = get_data_point_with_context_list(datapoint)
        defense = defender_model(compact_data)

        temp_data = {
            "Question": question,
            "Defense": defense
        }

        # Add to log list and checkpoint to JSONL file
        output_log.append(temp_data)
        with open(result_log_path, 'a', encoding='utf-8') as file:
            file.write(json.dumps(temp_data, ensure_ascii=False) + '\n')

        print(f"Data {i} saved")
        i += 1

    except Exception as e:
        print(f"Error processing question: {question}")
        print(f"Exception: {e}")

print("Processing complete.")


Data 0 saved
Data 1 saved


KeyboardInterrupt: 